In [3]:
import yfinance as yf
import pandas as pd
import os
import re
import pandas as pd
import pytz
import datetime
from datetime import datetime, timedelta
from ipywidgets import Button, VBox, Label, Textarea
from IPython.display import display


In [4]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import matplotlib.pyplot as plt

In [5]:
watchlists = {
    '9/5-9/9':["SBIT",
        "PSN",
        "RYAN",
        "SFM",
        "CNC",
        "EXAS",
        "ETHD",
        "NVDS",
        "VXZ",
        "WLFC",
        "KOLD",
        "GSHD",
        "VXX",
        "UVXY"],
    '9/12-9/16':[
        'PSN', 'CHRW','SHAK','ETHD','EXAS','VXZ','K','WLFC','GSHD','VXX'
    ],
    '9/14':[
        "NVMI", "DUOL", "CVNA", "SE", "FOUR", "NTRA",
        "AZPN", "IESC", "SHOP", "CAVA", "MNDY", "BMA",
        "KRUS", "PAM", "RARE", "LRN", "MASI", "ICUI",
        "FTNT", "AXON"
    ],
    '9/15':[
        "NVMI", "APP", "LLY", "Z", "TAYD", "SHOP", "NTRA", "DUOL",
        "IESC", "NRG", "SE", "MNDY", "CAVA", "FOUR", "BMA",
        "ICUI", "PAM", "MASI", "LRN", "RARE"
    ],
    '9/16':[
        "NVMI", "APP", "LLY", "Z", "TAYD", "SHOP", "NTRA", "DUOL", "IESC", "NRG",
        "SE", "MNDY", "CAVA", "FOUR", "BMA", "ICUI", "PAM", "MASI", "LRN", "RARE"
    ],
    '9/19':["TECL", "NVDL", "COHR", "APP", "TGLS", "BMA", "SE", "RXST", "HRB", "PAM", "SKY", "KRUS", "FOUR", "EXAS", "CAVA", "RARE", "TEM", "CLMB", "SBUX", "LOAR"],
    '9/20':["TQQQ", "TECL", "FIVE", "ROM", "TREE", "APP", "ACLX", "MRVL", "COHR", "PTF", "ZG", "Z", "BMA", "RXST", "NVDL", "HALO", "PAM", "SBUX", "HRB", "SE"]


}
CONFIG = {
    'rsi_window': 15,
    'stoch_rsi_window':15,
    'buy_rsi_threshold': 30,
    'capital': 25000,
    'target_percentage': 0.55,  
    'stop_loss_percentage': 0.5,#(-)
    'stage2tp':1,
    'stage2sl':0.5,
    'stage2':False,
    'activateS2': False,
    'reorder': True,
    'Trail': False,
    'folder_name': 'LiveDataTest',
    'watchlist': watchlists['9/20'],
    'start_time': datetime(2024, 9, 20, 11, 4),
    'test_date':'2024-08-20',
    'clicks':370,
    'lower-limit': -375,
    'trailing-loss':200,
    'trailing-breakthrough':300,
    'trailing': False,
    'trading': True,
    "TSL": 0,
    'TSL_D': 0.55
   
}

data_stats = {}
def main():
# Configuration Section
    global times, profits, exits
    profits = []
    times = []
    buys = []
    exits = []
    data_stats = {
        'DPH': False,
        'DSLH': False,
        'Profit': 0,
        'TT': 0,
        'Succ_T': 0,
        'Failed_T': 0,
        'TTR': 0,
        'WHF': 'NA',
        'High': 0,
        'Low': 0,
        'trailing-saved':0


    }
    BUY_STOCK = {
        'signal': False,
        'stock_name': 'AAPL',
        'quantity': 0,
        'current_price': 0,
        'target_price': 0,
        'stop_price': 0,
        'passcode': 123456
    }

    def get_most_volatile_stocks(num_stocks=5):
    # You can change the method of getting these stocks based on your requirements
        tickers = ['AAPL', 'TSLA', 'AMZN', 'GOOGL', 'MSFT', 'NFLX', 'NVDA', 'AMD']  # Example tickers
        volatility_data = []
        
        for ticker in tickers:
            try:
                stock_data = yf.download(ticker, period='1d', interval='1m')
                if not stock_data.empty:
                    stock_data['Returns'] = stock_data['Close'].pct_change()
                    volatility = stock_data['Returns'].std()  # Calculate volatility
                    volatility_data.append((ticker, volatility))
            except Exception as e:
                print(f"Failed to download data for {ticker}: {e}")

    # Sort by volatility and return the top N stocks
        volatility_data.sort(key=lambda x: x[1], reverse=True)
        return [ticker for ticker, _ in volatility_data[:num_stocks]]

    # Update the CONFIG['watchlist'] with the most volatile stocks
    def update_watchlist():
        most_volatile_stocks = get_most_volatile_stocks(num_stocks=5)
        CONFIG['watchlist'] = most_volatile_stocks
        print(f"Updated watchlist: {CONFIG['watchlist']}")

# Example CONFIG dictionary
    CONFIG = {
        'watchlist': [],
        'capital': 10000,
        'test_date': datetime.now().strftime('%Y-%m-%d'),
        'start_time': datetime.now(),
        'rsi_window': 15,
        'stoch_rsi_window':15,
        'buy_rsi_threshold': 30,
        'capital': 25000,
        'target_percentage': 0.55,  
        'stop_loss_percentage': 0.5,#(-)
        'stage2tp':1,
        'stage2sl':0.5,
        'stage2':False,
        'activateS2': False,
        'reorder': True,
        'Trail': False,
        'folder_name': 'LiveDataTest',
        'clicks':370,
        'lower-limit': -375,
        'trailing-loss':200,
        'trailing-breakthrough':300,
        'trailing': False,
        'trading': True,
        "TSL": 0,
        'TSL_D': 0.55
   
        # Add other config settings as needed
    }

    # Call the update_watchlist function to refresh the watchlist
    update_watchlist()


    #Stock Bot
    def open_webull():
        global driver
        options = Options()
        options.add_experimental_option("detach", True)
        options.add_argument("--no-sandbox")
        options.add_argument("--disable-dev-shm-usage")
        options.add_argument("--disable-gpu")
        options.add_argument("--remote-debugging-port=9222")


        # Set the path to your Chrome profile
        chrome_profile_path = r"C:\Users\danym\AppData\Local\Google\Chrome\User Data"
        chrome_profile_dir = "Profile 9"  # Change this if you are using a different profile name


        options.add_argument(f"user-data-dir={chrome_profile_path}")
        options.add_argument(f"profile-directory={chrome_profile_dir}")


        service = Service(ChromeDriverManager().install())
        driver = webdriver.Chrome(service=service, options=options)
        # Open the Webull paper trading page
        driver.get("https://app.webull.com/stocks")
        print("Opened Webull paper trading page.")
        print("Enter Passcode")
        input()


    def actually_purchase():
        global driver, BUY_STOCK
        # Wait for the BUY_NOW signal to be true
        while not BUY_STOCK['signal']:
            print("Waiting for BUY_NOW signal to be true...")
            time.sleep(5)  # Check every 5 seconds


        print("BUY_NOW signal received. Proceeding with stock purchase...")    
        # Clear the search box before entering the stock symbol
        search_box = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="app"]/div/form/span/div/span/input'))
        )
        #search_box.click()  # Click on the search box to focus
        #search_box.send_keys(Keys.CONTROL + 'a')  # Select all text (use 'COMMAND' on Mac)
        #search_box.send_keys(Keys.DELETE)  # Delete the selected text
        print("Cleared the search box.")
        # Find and click the stock symbol/name
        print("Searching for stock:", BUY_STOCK['stock_name'])
        search_box.click()
        search_box.send_keys(BUY_STOCK['stock_name'])
        search_box.send_keys(Keys.ENTER)
        print(f"Entered stock name '{BUY_STOCK['stock_name']}' and submitted search.")
        time.sleep(1)
        # Click Order Type and select Market
        print("Selecting Order Type...")
        order_type_button = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="Trade"]/div/div[2]/div[2]/div[2]/div[1]/div[2]'))
        )
        order_type_button.click()
        market_option = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '/html/body/div[4]/div/div[2]/div'))
        )
        print("Selecting Market option...")
        market_option.click()


        #Clear number of shares
        quantity_box = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="Trade"]/div/div[2]/div[2]/div[2]/div[2]/div[2]/div[1]/input'))
        )
        quantity_box.click()
        quantity_box.send_keys(Keys.CONTROL + 'a')  # Select all text (use 'COMMAND' on Mac)
        quantity_box.send_keys(Keys.DELETE)  # Delete the selected text
        print("Cleared the shares box.")
        # Enter the number of shares
        print("Entering quantity...")
        quantity_box.clear()
        quantity_box.send_keys(str(BUY_STOCK['quantity']))
        print(f"Entered quantity: {BUY_STOCK['quantity']}")


        # Check Stop loss Order
        stop_loss_checkbox = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="Trade"]/div/div[2]/div[4]/div/div[1]/div[1]/div/span[1]'))
        )
        print("Checking Stop Loss Order...")
        stop_loss_checkbox.click()


        # Check Take profit order
        print("Checking Take Profit Order...")
        take_profit_checkbox = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="Trade"]/div/div[2]/div[4]/div/div[2]/div[1]/div/span[1]'))
        )
        take_profit_checkbox.click()


        #Clear stop loss price
        stop_loss_price_box = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="Trade"]/div/div[2]/div[4]/div/div[1]/div[2]/div[1]/div[2]/div[1]/input'))
        )
        stop_loss_price_box.click()
        stop_loss_price_box.send_keys(Keys.CONTROL + 'a')
        stop_loss_price_box.send_keys(Keys.DELETE)
        # Enter stop loss price
        print("Entering Stop Loss Price...")
        stop_loss_price_box.clear()
        stop_loss_price_box.send_keys(str(BUY_STOCK['stop_price']))


        #Clear take profit price
        take_profit_price_box = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="Trade"]/div/div[2]/div[4]/div/div[2]/div[2]/div[1]/div[2]/div[1]/input'))
        )
        take_profit_price_box.click()
        take_profit_price_box.send_keys(Keys.CONTROL + 'a')
        take_profit_price_box.send_keys(Keys.DELETE)
        # Enter take profit price
        print("Entering Take Profit Price...")
        take_profit_price_box.clear()
        take_profit_price_box.send_keys(str(BUY_STOCK['target_price']))
       
        # Press “Buy Stock”
        final_buy_button = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="Trade"]/div/div[2]/div[5]/button'))
        )
        print("Pressing Buy Stock button...")
        #final_buy_button.click()
        print("Clicked Buy Stock button.")


        # Set BUY_NOW signal to false
        BUY_STOCK['signal'] = False
        print("Stock purchased successfully.")


   
    date_today = CONFIG['test_date']


    def get_stock_data(stock_symbol, date):
        start_date = date
        end_date = (pd.to_datetime(date) + pd.DateOffset(days=1)).strftime('%Y-%m-%d')
       
        # Fetch data at 5-minute intervals
        stock_data = yf.download(stock_symbol, start=start_date, end=end_date, interval='1m')
       
        # Ensure data is only for the specified day
        #stock_data = stock_data.loc[stock_data.index.date == pd.to_datetime(date).date()]
       
        return stock_data


    def get_data_recent():
        for symbol in CONFIG['watchlist']:
            data = get_stock_data(symbol, CONFIG['test_date'])
            file_path = os.path.join(CONFIG['folder_name'], f'{symbol}_1min_data_{date_today}.csv')
            data.to_csv(file_path)
            #print(f"Data for {symbol} on {CONFIG['test_date']} saved to {file_path}")
    def extract_ticker_from_filename(filename):
        ticker = filename.split('_')[0]
        return ticker
    #retrieve last period minutes
    def retrieve_data(ticker, time, test_date):
        folder_path = CONFIG['folder_name']
        files = os.listdir(folder_path)
        for file in files:
           
            if extract_ticker_from_filename(file) == ticker and test_date in file:
                data = pd.read_csv(os.path.join(folder_path, file))
                data['Datetime'] = pd.to_datetime(data['Datetime'])
                data['Datetime'] = data['Datetime'].dt.tz_localize(None)  # Remove timezone information
                data = data.set_index('Datetime')
               
                #ctime = ctime.replace(tzinfo=None)  # Ensure the time is timezone-naive
                data_slice = data.loc[time - pd.Timedelta(minutes=15):time]  # Return last 15 minutes of data
                return data_slice
        return pd.DataFrame()  # Return empty DataFrame if no data found
    #retrieve last period rows
    def retrieve_data_trail(ticker, time, test_date):
        #print("Retrieving data")
        #print("Ticker:", ticker)
        #print("Time:", time)
        #print("Test Date:", test_date)


        folder_path = CONFIG['folder_name']
        files = os.listdir(folder_path)
       
        found = False
        for file in files:
            if extract_ticker_from_filename(file) == ticker and test_date in file:
                #print("Extracting")
                data = pd.read_csv(os.path.join(folder_path, file))
                #print("Raw Data Head:")
                #print(data.head())
               
                data['Datetime'] = pd.to_datetime(data['Datetime'])
                data['Datetime'] = data['Datetime'].dt.tz_localize(None)  # Remove timezone information
                data = data.set_index('Datetime')
               
                #print("Data Index Head:")
                #print(data.index[:10])
               
                time = time.replace(tzinfo=None)  # Ensure the time is timezone-naive
                if CONFIG['Trail']:
                    if time in data.index:
                        # Select all rows up to the closest time
                        data_up_to_time = data.loc[:time]
                       
                        # Get the last 15 rows
                        data_slice = data_up_to_time.tail(15)
                        found = True
                    else:
                        #print('No trail found at:', time)
                        data_slice = pd.DataFrame()  # Return empty DataFrame if time is not in index
                else:
                    data_slice = data.loc[time - pd.Timedelta(minutes=15):time]  # Return last 15 minutes of data
                break
       
        #if not found:
            #print("No data found")
       
        #return data_slice


    def calculate_rsi(data, window=CONFIG['rsi_window']):
        delta = data['Open'].diff()
        gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
        #rs = gain / loss
        rs = gain / (loss)  # Adding a small constant to avoid division by zero
        return 100 - (100 / (1 + rs))


    def calculate_stochrsi(data, window=CONFIG['stoch_rsi_window']):
        min_val = data['Open'].rolling(window=window).min()
        max_val = data['Open'].rolling(window=window).max()
        return (data['Open'] - min_val) / (max_val - min_val)


    def check_buy(rsi):
        return rsi.iloc[-1] < CONFIG['buy_rsi_threshold']


    def buy_stock(ticker, price, capital):
        shares_bought = capital // price
        invested = price * shares_bought
        timestamp = datetime.now()
        BUY_STOCK['stock_name'] = ticker
        BUY_STOCK['current_price'] = price
        BUY_STOCK['quantity'] = shares_bought
        # Calculate target and stop prices
        BUY_STOCK['target_price'] = round(BUY_STOCK['current_price'] * (1 + (CONFIG['target_percentage']/100)), 1)
        BUY_STOCK['stop_price'] = round(BUY_STOCK['current_price'] * (1 - (CONFIG['stop_loss_percentage']/100)), 1)
        #BUY
       
        BUY_STOCK['signal'] = True
        #actually_purchase()
        #print(BUY_STOCK['stock_name'])
        #print(BUY_STOCK['current_price'])
        #print(BUY_STOCK['quantity'])
        #print(BUY_STOCK['signal'])
        return (f"{timestamp}: {shares_bought} shares of {ticker} purchased. "
                f"Price bought is {price}. Total invested is {invested}."), shares_bought, invested


    def check_conditions(ticker, init_price, final_price, shares, target_price, stop_loss, time):
        profit_loss = shares * (final_price - init_price)
        if CONFIG['activateS2']:
            target2 = init_price * (1 + CONFIG['stage2tp'] / 100)
            loss2 = init_price * (1 + CONFIG['stage2sl'] / 100)
            if CONFIG['stage2'] == False and final_price >= target_price:
                #data_stats['Succ_T'] += 1
                CONFIG['stage2'] = True
               
                return (f"Target price reached at {time}: {shares} shares of {ticker} sold. "
                        f"Price sold is {final_price}. Profit/Loss is {profit_loss}."), False, profit_loss
            elif CONFIG['stage2'] == False and  final_price <= stop_loss:
                data_stats['Failed_T'] += 1
                return (f"Stop loss reached at {time}: {shares} shares of {ticker} sold. "
                        f"Price sold is {final_price}. Profit/Loss is {profit_loss}."), True, profit_loss
            elif CONFIG['stage2'] and final_price >= target2:
                data_stats['Succ_T'] += 1
                CONFIG['stage2'] = False
                return (f"2Target price reached at {time}: {shares} shares of {ticker} sold. "
                        f"Price sold is {final_price}. Profit/Loss is {profit_loss}."), True, profit_loss
            elif CONFIG['stage2'] and final_price <= loss2:
                data_stats['Succ_T'] += 1
                CONFIG['stage2'] = False
                return (f"2LTarget price reached at {time}: {shares} shares of {ticker} sold. "
                        f"Price sold is {final_price}. Profit/Loss is {profit_loss}."), True, profit_loss
               
            else:
                return "Check Condition Failure", False, 0
        else:
            if final_price >= target_price:
                data_stats['Succ_T'] += 1
               
                return (f"Target price reached at {time}: {shares} shares of {ticker} sold. "
                        f"Price sold is {final_price}. Profit/Loss is {profit_loss}."), True, profit_loss
            elif final_price <= stop_loss:
                data_stats['Failed_T'] += 1
                return (f"Stop loss reached at {time}: {shares} shares of {ticker} sold. "
                        f"Price sold is {final_price}. Profit/Loss is {profit_loss}."), True, profit_loss
            else:
                return "Check Condition Failure", False, 0








    def monitor_stock(ticker, init_price, shares, target_price, stop_loss, time):
        output = ""
        while True:
            data = retrieve_data(ticker, time, CONFIG['test_date'])
            if not data.empty:
                current_price = data['Open'].iloc[-1]
                output += (f"Current price of {ticker} at {time}: {current_price}\n"
                        f"Purchase price of {ticker}: {init_price}\n")
                message, condition_met, profit_loss = check_conditions(ticker, init_price, current_price, shares, target_price, stop_loss, time)
                output += message + "\n"
                if condition_met:
                    exits.append(time)
                    return output, True, profit_loss
            time += timedelta(minutes=1)
            if time > data.index.max():
                break
        return output, False, 0




    def calculate_total_account_value(account_value, invested_stock, invested_price, invested_shares, time):
        total_value = account_value
        if invested_stock:
            data = retrieve_data(invested_stock, time, CONFIG['test_date'])
            if not data.empty:
                current_price = data['Open'].iloc[-1]
                total_value += invested_shares * (current_price - invested_price)
        return total_value


    def reorder_watchlist(ticker, success):
        if success:
            pass
            #NOTE: consider passing this condition to create a steadier win rate
            #nothing
            #print('Success but do not reorder')
            CONFIG['watchlist'].remove(ticker)
            CONFIG['watchlist'].append(ticker)
        else:
            pass
            CONFIG['watchlist'].remove(ticker)
            CONFIG['watchlist'].insert(0, ticker)


    """def trading_loop(time, current_stock, account_value, invested_stock, invested_price, invested_shares, target_price, stop_loss):
        minute_output = ""
        if invested_stock:
            data = retrieve_data(invested_stock, time, CONFIG['test_date'])
            if len(data) < 15:
                minute_output += f"Not enough data for {invested_stock} at {time}\n"
            else:
                current_price = data['Open'].iloc[-1]
                minute_output += f"Current price of {invested_stock} at {time}: {current_price}\n"
                minute_output += f"Purchase price of {invested_stock}: {invested_price}\n"
                monitor_output, condition_met, profit_loss = monitor_stock(invested_stock, invested_price, invested_shares, target_price, stop_loss, time)
                minute_output += monitor_output
                if condition_met:
                   
                    account_value += profit_loss
                    if CONFIG['reorder']:
                        reorder_watchlist(invested_stock, profit_loss > 0)  # Reorder based on trade outcome


                    # Reset investment details
                    invested_stock = None
                    invested_price = 0
                    invested_shares = 0
                    target_price = 0
                    stop_loss = 0
                    minute_output += f"Investment exited. Searching for new buy signal...\n"
                    return minute_output, True, account_value  # Return True to indicate investment exit
                else:
                    minute_output += f"Monitoring {invested_stock} at {time}\n"
                    return minute_output, False, account_value
        else:
            # No stock is currently invested
            minute_output += "No active investment\n"
            return minute_output, False, account_value"""

    def trading_loop(time, current_stock, account_value, invested_stock, invested_price, invested_shares, target_price, stop_loss):
        minute_output = ""
        if invested_stock:
            try:
                data = retrieve_data(invested_stock, time, CONFIG['test_date'])
                if data.empty or len(data) < 15:
                    minute_output += f"Not enough data for {invested_stock} at {time}\n"
                    return minute_output, False, account_value
                
                current_price = data['Open'].iloc[-1]
                minute_output += f"Current price of {invested_stock} at {time}: {current_price}\n"
                minute_output += f"Purchase price of {invested_stock}: {invested_price}\n"
                monitor_output, condition_met, profit_loss = monitor_stock(invested_stock, invested_price, invested_shares, target_price, stop_loss, time)
                minute_output += monitor_output
                
                if condition_met:
                    account_value += profit_loss
                    if CONFIG['reorder']:
                        reorder_watchlist(invested_stock, profit_loss > 0)

                    # Reset investment details
                    invested_stock = None
                    invested_price = 0
                    invested_shares = 0
                    target_price = 0
                    stop_loss = 0
                    minute_output += f"Investment exited. Searching for new buy signal...\n"
                    return minute_output, True, account_value

                else:
                    minute_output += f"Monitoring {invested_stock} at {time}\n"
                    return minute_output, False, account_value

            except Exception as e:
                minute_output += f"Error retrieving data for {invested_stock}: {str(e)}\n"
                return minute_output, False, account_value
        else:
            minute_output += "No active investment\n"
            return minute_output, False, account_value



    def bot():
        #nonlocal buys, profits, exits
        capital = CONFIG['capital']
        account_value = capital
        ctime = CONFIG['start_time']  # Start time from config
        total_trades = 0
        test_date = CONFIG['test_date']  # Specify the date you're testing on


        # Variables to track the current investment
        invested_stock = None
        invested_price = 0
        invested_shares = 0
        target_price = 0
        stop_loss = 0
        total_value = 0


        output_widget = Textarea(value="", layout={'width': '100%', 'height': '300px'})
       
        def on_button_click(b):
            nonlocal test_date, ctime, invested_stock, invested_price, invested_shares, target_price, stop_loss, account_value, total_trades
            #nonlocal profits, times
            minute_output = ""


            if invested_stock:
                result = trading_loop(ctime, invested_stock, account_value, invested_stock, invested_price, invested_shares, target_price, stop_loss)
                if result:
                    minute_output, exited, account_value = result
                    if exited:
                        # Increment total_trades when exiting a position
                        total_trades += 1
                        # Restart trading loop after exiting position
                        invested_stock = None
                        invested_price = 0
                        invested_shares = 0
                        target_price = 0
                        stop_loss = 0
                else:
                    minute_output = "Error: trading_loop did not return a valid result.\n"
            else:
                # Process new investments
                minute_output = ""
                for ticker in CONFIG['watchlist']:
                    minute_output += f"Processing {ticker} at {ctime}\n"
                    data = retrieve_data(ticker, ctime, test_date)
                    if len(data) < 15:
                        minute_output += f"Not enough data for {ticker} at {ctime}\n"
                        continue  # Skip if there's not enough data for RSI calculation
                    rsi = calculate_rsi(data)
                    stochrsi = calculate_stochrsi(data)
                    if check_buy(rsi):
                        buys.append(ctime)
                        minute_output += f"Detected buy signal for {ticker} at {ctime}\n"
                        message, shares, invested = buy_stock(ticker, data['Open'].iloc[-1], capital)
                        minute_output += message + "\n"
                        invested_stock = ticker
                        invested_price = data['Open'].iloc[-1]
                        invested_shares = shares
                        target_price = invested_price * (1 + CONFIG['target_percentage'] / 100)  # Target price percentage above purchase
                        stop_loss = invested_price * (1 - CONFIG['stop_loss_percentage'] / 100)  # Stop loss percentage below purchase
                        break  # Exit loop after making an investment
                    else:
                        minute_output += f"No buy signal found for {ticker} at {ctime}\n"
           
            # Calculate and display the total account value
            total_value = calculate_total_account_value(account_value, invested_stock, invested_price, invested_shares, ctime)
           
            if (total_value >= CONFIG['capital'] * (1.01)):
                data_stats['DPH'] = True
                if (data_stats['DSLH'] == False):
                    data_stats['WHF'] = 'Profit'
            if (total_value <= CONFIG['capital'] * (0.99)):
                data_stats['DSLH'] = True
                if (data_stats['DPH'] == False):
                    data_stats['WHF'] = 'Loss'
           
            if total_trades > 0:
                round(total_value, 2)
                data_stats['Profit'] = total_value - CONFIG['capital']
                data_stats['Profit'] = round(data_stats['Profit'], 2)


                data_stats['TTR'] = data_stats['Succ_T'] / total_trades
                data_stats['TTR'] = round(data_stats['TTR'], 3)
                if data_stats['Profit'] > data_stats['High']:
                    data_stats['High'] = data_stats['Profit']
                if data_stats['Profit'] < data_stats['Low']:
                    data_stats['Low'] = data_stats['Profit']
                minute_output += f"Total account value at {ctime}: {total_value}\nEarnings: {data_stats['Profit']}\nHigh: {data_stats['High']}\nLow: {data_stats['Low']}\nDaily Profit Hit: {data_stats['DPH']}\nDaily Stop-Loss Hit: {data_stats['DSLH']}\nTotal Trades: {total_trades}\nTotal Trade Success Rate: {data_stats['TTR']}\nWHF: {data_stats['WHF']}\nSaved at: {data_stats['trailing-saved']}"
            else:
                minute_output += f"No trades today"
            output_widget.value = minute_output
            print(minute_output)  # Output to console
           
            profits.append(data_stats['Profit'])
            times.append(ctime)
            ctime += timedelta(minutes=1)
            time_label.value = f"Current time: {ctime.strftime('%Y-%m-%d %H:%M')}"
       
        button = Button(description="Run Bot")
        button.on_click(on_button_click)
       
        time_label = Label(value=f"Current time: {ctime.strftime('%Y-%m-%d %H:%M')}")
       
        display(VBox([button, time_label, output_widget]))


        def automate_clicks(num_clicks):
            for _ in range(num_clicks):
                on_button_click(None)
        count = 0
        while CONFIG['trading'] and count < CONFIG['clicks']:
            if CONFIG['trailing']:
                if data_stats['Profit'] <= data_stats['High'] - CONFIG['trailing-loss']:
                    data_stats['trailing-saved'] = data_stats['Profit']
                    #CONFIG['trading'] = False
                    CONFIG['trailing'] = False


            if data_stats['Profit'] >= CONFIG['trailing-breakthrough']:
                CONFIG['trailing'] = True
            if data_stats['Profit'] <= CONFIG['lower-limit']:
                pass
                #CONFIG['trading'] = False
            time.sleep(60)
            get_data_recent()
            automate_clicks(1)
            count += 1
    #open_webull()


    get_data_recent()
    bot()
    # Example usage
    def graph_data():
        # Create the plot
        plt.figure(figsize=(10, 6))


        # Plot the blue profit line with a lower zorder
        plt.plot(times, profits, marker='o', linestyle='-', color='b', label='Profit', zorder=1)


        # Mark buy signals in red on top of the blue line with a higher zorder
        buy_profits = [profits[times.index(buy_time)] for buy_time in buys if buy_time in times]
        plt.scatter(buys, buy_profits, color='g', marker='x', s=100, label='Buy Signal', zorder=2)
       
        exit_marks = [profits[times.index(exit_time)] for exit_time in exits if exit_time in times]
        plt.scatter(exits, exit_marks, color='r', marker='x', s=100, label='Exit Signal', zorder=2)


        # Add title and labels
        plt.title('Earnings Over Time for ' + CONFIG['test_date'])
        plt.xlabel('Time')
        plt.ylabel('Profit')


        # Set initial limits
        lower_limit = -1000
        upper_limit = 1000
        plt.ylim(lower_limit, upper_limit)


        # Update limits based on actual data
        min_profit = min(profits)
        max_profit = max(profits)


        if min_profit < lower_limit:
            plt.ylim(min_profit, plt.ylim()[1])
        if max_profit > upper_limit:
            plt.ylim(plt.ylim()[0], max_profit)


        # Show grid
        plt.grid(True)
       
        # Add legend
        plt.legend()


        # Display the plot
        plt.show()
    #graph_data()
main()


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

1 Failed download:
['TSLA']: YFPricesMissingError('$%ticker%: possibly delisted; no price data found  (1m 2024-10-01 -> 2024-10-02)')
[*********************100%***********************]  1 of 1 completed

1 Failed download:
['NVDA']: YFPricesMissingError('$%ticker%: possibly delisted; no price data found  (1m 2024-10-01 -> 2024-10-02)')


Updated watchlist: ['TSLA', 'NVDA', 'AMD', 'NFLX', 'AMZN']


[*********************100%***********************]  1 of 1 completed

1 Failed download:
['AMD']: YFPricesMissingError('$%ticker%: possibly delisted; no price data found  (1m 2024-10-01 -> 2024-10-02)')
[*********************100%***********************]  1 of 1 completed

1 Failed download:
['NFLX']: YFPricesMissingError('$%ticker%: possibly delisted; no price data found  (1m 2024-10-01 -> 2024-10-02)')
[*********************100%***********************]  1 of 1 completed

1 Failed download:
['AMZN']: YFPricesMissingError('$%ticker%: possibly delisted; no price data found  (1m 2024-10-01 -> 2024-10-02)')


KeyboardInterrupt: 